---
    title: 'The Rise of Python in Data Product Development: Exploring Solara'
    author: Ítalo Epifânio
    date: 10/17/2023
    categories: [English]
    image: rising-python.png
    draft: false
    execute: 
        enabled: false
        error: false
        echo: false
        freeze: true
---

## Your Data, Your Language, Your Products

The emerging trend of tool-building for data scientists is manifestly 
by the immense traction PyScript gained within the tech community upon its initiation.
This momentum shows a collective desire to simplify the journey of creating data products. 
Data scientists, that already posses a wide range of knowledge, can accelerate the development 
of data applications without diving into a new ecosystem like Javascript.

As advocates of Python for data product development, we're always observing the the 
evolution of its ecossystem. At Palaimon GmbH, our expertise with [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/) (a lib with 
interactive HTML widgets for Jupyter Notebooks) has led to innovations like [Ipyannotator](https://github.com/palaimon/ipyannotator), 
an open source framework dedicated to data annotation, but using ipywidgets has some 
challenges and as the applications grows in complexity, they became increasingly difficult t
o manage.

Solara, an open-source framework, draws inspiration from the JavaScript ecosystem to create reusable components using a strategy that has stood the test of time. Our experiments with Solara have demonstrated its capability to facilitate the crafting of larger and more intricate data applications

::: {.callout-note icon=false}
# Python + Data + Web Apps

If you've ever felt limited by a lack of JavaScript experience or wished for a 
more straightforward way to build data products, Python might be your answer. 
With tools available in Python, building data products can be more aligned with 
what you already know. 
:::

## Solara

Solara enables the crafting of expansive data applications by incorporating concepts from React. Its declarative code style makes the codebase more concise, reduces errors, and simplifies comprehension. Solara's design facilitates easy code reuse through componentization, feature that was ambiguous when developing with ipywidgets.

While ipywidgets are commonly paired with the [voilà library](https://github.com/voila-dashboards/voila) to convert Jupyter notebooks into web applications for non-technical users, Solara boasts its own web server. This allows for the sharing of a more tailored web application compared to voilà.

In my exploration of Solara for video and image annotation, I discovered that its built-in plotly component lacked the capabilities to construct an annotator. However, [I successfully developed a custom plotly component](https://github.com/widgetti/solara/pull/285) and was able to contribute with [the new feature](https://solara.dev/examples/visualization/annotator). I'm immensely satisfied with using Solara for such tool development and the ease of reusing its components.

Below is an example of the annotator feature integrated into the library:

In [ ]:
import json

import solara
import numpy as np
from PIL import Image as PILImage
import plotly.graph_objects as go

In [ ]:
def generate_circle_image(image_size: int) -> PILImage:
    circle_image = np.ones((image_size, image_size, 3)) * 255

    x_center, y_center = image_size//2, image_size//2
    radius = image_size//5

    y, x = np.ogrid[-y_center:image_size - y_center, -x_center:image_size - x_center]

    mask = x**2 + y**2 <= radius**2

    circle_image[mask] = [0, 0, 255]
    circle_image = circle_image.astype(np.uint8)

    return PILImage.fromarray(circle_image)

In [ ]:
class CustomEncoder(json.JSONEncoder):
    """
    Custom JSON encoder for Plotly objects.

    Plotly may return objects that the standard JSON encoder can't handle. This
    encoder converts such objects to str, allowing serialization by json.dumps
    """

    def default(self, o):
        if isinstance(o, object):
            return str(o)
        return super().default(o)

In [ ]:
@solara.component
def ImageAnnotator(image: np.ndarray):
    shapes = solara.use_reactive(None)

    def on_relayout(data):
        if data is None:
            return

        relayout_data = data["relayout_data"]

        if "shapes" in relayout_data:
            shapes.value = relayout_data["shapes"]

    fig = go.FigureWidget(
        layout=go.Layout(
            width=image.size[0],
            height=image.size[1],
            xaxis={
                'range': [0, image.size[0]],
                'showgrid': False,
            },
            yaxis={
                'range': [0, image.size[1]],
                'showgrid': False,
            },
            showlegend=False,
            autosize=False,
            margin={'t': 0, 'b': 0, 'l': 0, 'r': 0},
            modebar={
                "add": [
                    "drawclosedpath",
                    "eraseshape",
                ]
            },
        )
    )

    fig.add_layout_image(
        source=image,
        x=0,
        sizex=image.size[0],
        y=image.size[1],
        sizey=image.size[1],
        xref="x",
        yref="y",
        opacity=1.0,
        layer="below",
        sizing="stretch",
    )

    with solara.Row(gap='10px'):
        solara.FigurePlotly(fig, on_relayout=on_relayout)
        if not shapes.value:
            solara.Markdown("## Draw on the canvas")
        else:
            with solara.Column(gap='10px'):
                solara.Markdown("## Data returned by drawing")
                formatted_shapes = str(json.dumps(shapes.value, indent=2, cls=CustomEncoder))
                solara.Preformatted(formatted_shapes)

In [ ]:
circle_image = generate_circle_image(400)

ImageAnnotator(circle_image)

{{< video solara-example.mp4 >}}

## Navigating the Future with Python

In the ever-evolving landscape of data science and application development, tools like Solara are paving the way for more efficient and user-friendly solutions. By bridging the gap between Python and web applications, it offers a promising avenue for data scientists to leverage their expertise without the need to venture into unfamiliar territories. At Palaimon GmbH, we remain committed to harnessing the potential of such innovations, ensuring that the data science community has the best tools at their disposal.